### requirements for the following codings


In [ ]:
### packages required 
!pip install fair-esm 
!pip install torch
!pip install tensorflow
!pip install sklearn

### peptide embeddings with esm2_t6_8M_UR50D pretrained models
6 layers, 8M parameters, dataset: UR50/D 2021_04, embedding dimension: 320
mode download URL: https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t6_8M_UR50D.pt

In [ ]:
def esm_embeddings(peptide_sequence_list):
  # NOTICE: ESM for embeddings is quite RAM usage, if your sequence is too long, 
  #         or you have too many sequences for transformation in a single converting, 
  #         you conputer might automatically kill the job.
  import torch
  import esm
  import collections
  # load the model
  # NOTICE: if the model was not downloaded in your local environment, it will automatically download it.
  model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
  batch_converter = alphabet.get_batch_converter()
  model.eval()  # disables dropout for deterministic results

  # load the peptide sequence list into the bach_converter
  batch_labels, batch_strs, batch_tokens = batch_converter(peptide_sequence_list)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
  ## batch tokens are the embedding results of the whole data set

  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      # Here we export the last layer of the EMS model output as the representation of the peptides
      # model'esm2_t6_8M_UR50D' only has 6 layers, and therefore repr_layers parameters is equal to 6
      results = model(batch_tokens, repr_layers=[6], return_contacts=True)  
  token_representations = results["representations"][6]

  # Generate per-sequence representations via averaging
  # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
  sequence_representations = []
  for i, tokens_len in enumerate(batch_lens):
      sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
  # save dataset
  # sequence_representations is a list and each element is a tensor
  embeddings_results = collections.defaultdict(list)
  for i in range(len(sequence_representations)):
      # tensor can be transformed as numpy sequence_representations[0].numpy() or sequence_representations[0].to_list
      each_seq_rep = sequence_representations[i].tolist()
      for each_element in each_seq_rep:
          embeddings_results[i].append(each_element)
  embeddings_results = pd.DataFrame(embeddings_results).T
  return embeddings_results

### data loading and embeddings

In [1]:
import numpy as np
import pandas as pd

In [2]:
# training dataset loading
dataset = pd.read_excel('bitter_train.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence'] 
peptide_sequence_list = []
for seq in sequence_list:
    format_seq = [seq,seq] # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information

# employ ESM model for converting and save the converted data in csv format
embeddings_results = esm_embeddings(peptide_sequence_list)
embeddings_results.to_csv('bitter_train_esm2_t6_8M_UR50D_unified_320_dimension.csv')

# loading the y dataset for model development 
y_train = dataset['label']
y_train = np.array(y_train) # transformed as np.array for CNN model

In [3]:
# test dataset loading
dataset = pd.read_excel('bitter_test.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence'] 
peptide_sequence_list = []
for seq in sequence_list:
    format_seq = [seq,seq] # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information

# employ ESM model for converting and save the converted data in csv format
embeddings_results = esm_embeddings(peptide_sequence_list)
embeddings_results.to_csv('bitter_test_esm2_t6_8M_UR50D_unified_320_dimension.csv')

# loading the y dataset for model development 
y_test = dataset['label']
y_test = np.array(y_test) # transformed as np.array for CNN model

In [4]:
# assign the dataset 
X_train_data_name = 'bitter_train_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_train_data = pd.read_csv(X_train_data_name,header=0, index_col = 0,delimiter=',')

X_test_data_name = 'bitter_test_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_test_data = pd.read_csv(X_test_data_name,header=0, index_col = 0,delimiter=',')

X_train = np.array(X_train_data)
X_test = np.array(X_test_data)

# normalize the X data range
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train) # normalize X to 0-1 range 
X_test = scaler.transform(X_test)

In [5]:
# check the dimension of the dataset before model development
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(512, 320)
(128, 320)
(512,)
(128,)


### Model architecture

In [7]:
def ESM_CNN(X_train, y_train, X_test, y_test):
  from keras.layers import Input,InputLayer, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D,Conv1D
  from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, AveragePooling1D, MaxPooling1D
  from keras.models import Sequential,Model
  from keras.optimizers import SGD
  from keras.callbacks import ModelCheckpoint,LearningRateScheduler, EarlyStopping
  import keras
  from keras import backend as K
  inputShape=(320,1)
  input = Input(inputShape)
  x = Conv1D(128,(3),strides = (1),name='layer_conv1',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool1',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Conv1D(32,(3),strides = (1),name='layer_conv2',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Flatten()(x)
  x = Dense(64,activation = 'relu',name='fc1')(x)
  x = Dropout(0.15)(x)
  x = Dense(2,activation = 'softmax',name='fc2')(x)
  model = Model(inputs = input,outputs = x,name='Predict')
  # define SGD optimizer
  momentum = 0.5
  sgd = SGD(lr=0.01, momentum=momentum, decay=0.0, nesterov=False)
  # compile the model
  model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
  # learning deccay setting
  import math
  def step_decay(epoch): # gradually decrease the learning rate 
      initial_lrate=0.1
      drop=0.6
      epochs_drop = 3.0
      lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
            math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
      return lrate
  lrate = LearningRateScheduler(step_decay)

  # early stop setting
  early_stop = EarlyStopping(monitor='val_loss', patience = 40,restore_best_weights = True)

  # summary the callbacks_list
  callbacks_list = [ lrate , early_stop]

  model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                            epochs=200,callbacks=callbacks_list,batch_size = 8, verbose=1)
  return model, model_history

### 10-fold cross validation

In [15]:
#Implementing 10-fold cross validation
from sklearn.model_selection import KFold
k = 10 
kf = KFold(n_splits=k, shuffle = True, random_state=1)
X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)

# result collection list
ACC_collecton = []
BACC_collecton = []
Sn_collecton = []
Sp_collecton = []
MCC_collecton = []
AUC_collecton = []

for train_index , test_index in kf.split(y_train):
    X_train_CV , X_valid_CV = X_train.iloc[train_index,:],X_train.iloc[test_index,:]
    y_train_CV , y_valid_CV = y_train.iloc[train_index] , y_train.iloc[test_index]
    model, model_history = ESM_CNN(X_train_CV, y_train_CV, X_valid_CV, y_valid_CV)
    # confusion matrix 
    predicted_class= []
    predicted_protability = model.predict(X_valid_CV,batch_size=1)
    for i in range(predicted_protability.shape[0]):
      index = np.where(predicted_protability[i] == np.amax(predicted_protability[i]))[0][0]
      predicted_class.append(index)
    predicted_class = np.array(predicted_class)
    y_true = y_valid_CV    
    from sklearn.metrics import confusion_matrix
    import math
    # np.ravel() return a flatten 1D array
    TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
    ACC = (TP+TN)/(TP+TN+FP+FN)
    ACC_collecton.append(ACC)
    Sn_collecton.append(TP/(TP+FN))
    Sp_collecton.append(TN/(TN+FP))
    MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
    MCC_collecton.append(MCC)
    BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
    from sklearn.metrics import roc_auc_score
    AUC = roc_auc_score(y_valid_CV, predicted_protability[:,1])
    AUC_collecton.append(AUC)


Epoch 1/200


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


58/58 [==============================] - 1s 10ms/step - loss: 1.1301 - accuracy: 0.6413 - val_loss: 0.6766 - val_accuracy: 0.5962 - lr: 0.1000
Epoch 2/200
58/58 [==============================] - 0s 7ms/step - loss: 0.5806 - accuracy: 0.7152 - val_loss: 0.6782 - val_accuracy: 0.5962 - lr: 0.1000
Epoch 3/200
58/58 [==============================] - 0s 7ms/step - loss: 0.4001 - accuracy: 0.8043 - val_loss: 0.6760 - val_accuracy: 0.5962 - lr: 0.0600
Epoch 4/200
58/58 [==============================] - 0s 8ms/step - loss: 0.3908 - accuracy: 0.8435 - val_loss: 0.6737 - val_accuracy: 0.5962 - lr: 0.0600
Epoch 5/200
58/58 [==============================] - 0s 8ms/step - loss: 0.3087 - accuracy: 0.8630 - val_loss: 0.6661 - val_accuracy: 0.5962 - lr: 0.0600
Epoch 6/200
58/58 [==============================] - 0s 8ms/step - loss: 0.2574 - accuracy: 0.8804 - val_loss: 0.6123 - val_accuracy: 0.7885 - lr: 0.0360
Epoch 7/200
58/58 [==============================] - 0s 8ms/step - loss: 0.2460 - accur

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


58/58 [==============================] - 1s 10ms/step - loss: 1.5274 - accuracy: 0.6500 - val_loss: 0.7268 - val_accuracy: 0.4231 - lr: 0.1000
Epoch 2/200
58/58 [==============================] - 0s 8ms/step - loss: 0.5196 - accuracy: 0.7522 - val_loss: 0.8334 - val_accuracy: 0.4231 - lr: 0.1000
Epoch 3/200
58/58 [==============================] - 0s 8ms/step - loss: 0.4733 - accuracy: 0.7848 - val_loss: 0.7730 - val_accuracy: 0.4231 - lr: 0.0600
Epoch 4/200
58/58 [==============================] - 0s 8ms/step - loss: 0.3799 - accuracy: 0.8261 - val_loss: 0.7007 - val_accuracy: 0.4808 - lr: 0.0600
Epoch 5/200
58/58 [==============================] - 0s 8ms/step - loss: 0.3747 - accuracy: 0.8239 - val_loss: 0.6144 - val_accuracy: 0.6346 - lr: 0.0600
Epoch 6/200
58/58 [==============================] - 0s 8ms/step - loss: 0.2798 - accuracy: 0.8652 - val_loss: 0.4838 - val_accuracy: 0.7500 - lr: 0.0360
Epoch 7/200
58/58 [==============================] - 0s 7ms/step - loss: 0.2772 - accur

...

58/58 [==============================] - 1s 10ms/step - loss: 1.0064 - accuracy: 0.6009 - val_loss: 0.7345 - val_accuracy: 0.4510 - lr: 0.1000
Epoch 2/200
58/58 [==============================] - 0s 7ms/step - loss: 0.5676 - accuracy: 0.7245 - val_loss: 0.8266 - val_accuracy: 0.4510 - lr: 0.1000
Epoch 3/200
58/58 [==============================] - 0s 8ms/step - loss: 0.5321 - accuracy: 0.7137 - val_loss: 0.7910 - val_accuracy: 0.4510 - lr: 0.0600
Epoch 4/200
58/58 [==============================] - 0s 8ms/step - loss: 0.4503 - accuracy: 0.8026 - val_loss: 0.7923 - val_accuracy: 0.4510 - lr: 0.0600
Epoch 5/200
58/58 [==============================] - 0s 8ms/step - loss: 0.4011 - accuracy: 0.8200 - val_loss: 0.7034 - val_accuracy: 0.4706 - lr: 0.0600
Epoch 6/200
58/58 [==============================] - 0s 8ms/step - loss: 0.3192 - accuracy: 0.8590 - val_loss: 0.6879 - val_accuracy: 0.4902 - lr: 0.0360
Epoch 7/200
58/58 [==============================] - 0s 8ms/step - loss: 0.2683 - accur

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


58/58 [==============================] - 1s 11ms/step - loss: 1.0531 - accuracy: 0.5987 - val_loss: 0.7340 - val_accuracy: 0.4314 - lr: 0.1000
Epoch 2/200
58/58 [==============================] - 0s 8ms/step - loss: 0.5838 - accuracy: 0.7440 - val_loss: 0.8305 - val_accuracy: 0.4314 - lr: 0.1000
Epoch 3/200
58/58 [==============================] - 0s 8ms/step - loss: 0.4428 - accuracy: 0.8048 - val_loss: 0.7971 - val_accuracy: 0.4314 - lr: 0.0600
Epoch 4/200
58/58 [==============================] - 0s 8ms/step - loss: 0.4090 - accuracy: 0.8178 - val_loss: 0.7955 - val_accuracy: 0.4314 - lr: 0.0600
Epoch 5/200
58/58 [==============================] - 0s 8ms/step - loss: 0.3921 - accuracy: 0.8243 - val_loss: 0.7657 - val_accuracy: 0.4314 - lr: 0.0600
Epoch 6/200
58/58 [==============================] - 0s 8ms/step - loss: 0.2926 - accuracy: 0.8807 - val_loss: 0.6584 - val_accuracy: 0.5294 - lr: 0.0360
Epoch 7/200
58/58 [==============================] - 0s 8ms/step - loss: 0.2520 - accur

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


58/58 [==============================] - 1s 11ms/step - loss: 1.2954 - accuracy: 0.6356 - val_loss: 0.7135 - val_accuracy: 0.4902 - lr: 0.1000
Epoch 2/200
58/58 [==============================] - 0s 8ms/step - loss: 0.5347 - accuracy: 0.7549 - val_loss: 0.7323 - val_accuracy: 0.4902 - lr: 0.1000
Epoch 3/200
58/58 [==============================] - 1s 9ms/step - loss: 0.3804 - accuracy: 0.8351 - val_loss: 0.7662 - val_accuracy: 0.4902 - lr: 0.0600
Epoch 4/200
58/58 [==============================] - 0s 8ms/step - loss: 0.3705 - accuracy: 0.8460 - val_loss: 0.7799 - val_accuracy: 0.4902 - lr: 0.0600
Epoch 5/200
58/58 [==============================] - 0s 8ms/step - loss: 0.3014 - accuracy: 0.8872 - val_loss: 0.6828 - val_accuracy: 0.5294 - lr: 0.0600
Epoch 6/200
58/58 [==============================] - 0s 8ms/step - loss: 0.2609 - accuracy: 0.8937 - val_loss: 0.6065 - val_accuracy: 0.5882 - lr: 0.0360
Epoch 7/200
58/58 [==============================] - 0s 8ms/step - loss: 0.2250 - accur

In [16]:
from statistics import mean, stdev
print(mean(ACC_collecton),'±',stdev(ACC_collecton))
print(mean(BACC_collecton),'±',stdev(BACC_collecton))
print(mean(Sn_collecton),'±',stdev(Sn_collecton))
print(mean(Sp_collecton),'±',stdev(Sp_collecton))
print(mean(MCC_collecton),'±',stdev(MCC_collecton))
print(mean(AUC_collecton),'±',stdev(AUC_collecton))


0.8436274509803922 ± 0.046354042261516765
0.8451833578903087 ± 0.04535267386942948
0.8472818665252876 ± 0.062383802203329515
0.8430848492553298 ± 0.06595380921003595
0.6870940673431497 ± 0.0897090661409879
0.9147550263941262 ± 0.03135277887354698


### model evaluation in test dataset

In [11]:
# result collection list
ACC_collecton = []
BACC_collecton = []
Sn_collecton = []
Sp_collecton = []
MCC_collecton = []
AUC_collecton = []
model, model_history = ESM_CNN(X_train, y_train, X_test , y_test)
# confusion matrix 
predicted_class= []
predicted_protability = model.predict(X_test,batch_size=1)
for i in range(predicted_protability.shape[0]):
  index = np.where(predicted_protability[i] == np.amax(predicted_protability[i]))[0][0]
  predicted_class.append(index)
predicted_class = np.array(predicted_class)
y_true = y_test    
from sklearn.metrics import confusion_matrix
# np.ravel() return a flatten 1D array
TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
ACC = (TP+TN)/(TP+TN+FP+FN)
ACC_collecton.append(ACC)
Sn_collecton.append(TP/(TP+FN))
Sp_collecton.append(TN/(TN+FP))
MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
MCC_collecton.append(MCC)
BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
from sklearn.metrics import roc_auc_score
AUC = roc_auc_score(y_test, predicted_protability[:,1])
AUC_collecton.append(AUC)

Epoch 1/200


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


64/64 [==============================] - 1s 11ms/step - loss: 1.2182 - accuracy: 0.6699 - val_loss: 0.6945 - val_accuracy: 0.5000 - lr: 0.1000
Epoch 2/200
64/64 [==============================] - 0s 7ms/step - loss: 0.4414 - accuracy: 0.8047 - val_loss: 0.6954 - val_accuracy: 0.5000 - lr: 0.1000
Epoch 3/200
64/64 [==============================] - 1s 8ms/step - loss: 0.3617 - accuracy: 0.8320 - val_loss: 0.6930 - val_accuracy: 0.5000 - lr: 0.0600
Epoch 4/200
64/64 [==============================] - 1s 8ms/step - loss: 0.2810 - accuracy: 0.8867 - val_loss: 0.6908 - val_accuracy: 0.5078 - lr: 0.0600
Epoch 5/200
64/64 [==============================] - 1s 8ms/step - loss: 0.2524 - accuracy: 0.8809 - val_loss: 0.5629 - val_accuracy: 0.8359 - lr: 0.0600
Epoch 6/200
64/64 [==============================] - 1s 8ms/step - loss: 0.2159 - accuracy: 0.9043 - val_loss: 0.4176 - val_accuracy: 0.8750 - lr: 0.0360
Epoch 7/200
64/64 [==============================] - 0s 7ms/step - loss: 0.1810 - accur

In [12]:
print(ACC_collecton[0])
print(BACC_collecton[0])
print(Sn_collecton[0])
print(Sp_collecton[0])
print(MCC_collecton[0])
print(AUC_collecton[0])

0.9375
0.9379276637341154
0.9242424242424242
0.9516129032258065
0.8754275592730114
0.982421875


In [13]:
model.save('bitter_tensorflow_model',save_format = 'tf') 
!zip -r /content/bitter_tensorflow_model.zip /content/bitter_tensorflow_model

  adding: content/bitter_tensorflow_model/ (stored 0%)
  adding: content/bitter_tensorflow_model/variables/ (stored 0%)
  adding: content/bitter_tensorflow_model/variables/variables.index (deflated 64%)
  adding: content/bitter_tensorflow_model/variables/variables.data-00000-of-00001 (deflated 39%)
  adding: content/bitter_tensorflow_model/saved_model.pb (deflated 88%)
  adding: content/bitter_tensorflow_model/assets/ (stored 0%)
  adding: content/bitter_tensorflow_model/keras_metadata.pb (deflated 89%)
